In [12]:
import os
from collections import defaultdict
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [3]:
f = "net_202001_weekday.csv"
df_res1 = pd.read_csv(os.path.join("../", "mapequation", f), index_col=0)
df_res1.head()

,clusters,flow,node,ori_index,level_1,level_2
ind,,,,,,
0,1:1,0.023149,TAMPINES EAST,40,1,1_1
1,1:2,0.017636,BEDOK NORTH,16,1,1_2
2,1:3,0.013683,TOA PAYOH CENTRAL,68,1,1_3
3,1:4,0.013282,ALJUNIED,21,1,1_4
4,1:5,0.012734,TAMPINES WEST,78,1,1_5


In [4]:
f = "net_202001_weekend.csv"
df_res2 = pd.read_csv(os.path.join("../", "mapequation", f), index_col=0)
df_res2.head()

,clusters,flow,node,ori_index,level_1,level_2
ind,,,,,,
0,1:1,0.015337,ALJUNIED,21,1,1_1
1,1:2,0.013425,GEYLANG EAST,10,1,1_2
2,1:3,0.010406,BOULEVARD,79,1,1_3
3,1:4,0.009332,MARITIME SQUARE,38,1,1_4
4,1:5,0.007980,BALESTIER,108,1,1_5


In [81]:
df_merge = pd.merge(df_res1[["node", "level_1"]], df_res2[["node", "level_1"]], left_on="node", right_on="node", suffixes=["_weekday", "_weekend"])
df_merge.head()

,node,level_1_weekday,level_1_weekend
0,TAMPINES EAST,1,2
1,BEDOK NORTH,1,2
2,TOA PAYOH CENTRAL,1,2
3,ALJUNIED,1,1
4,TAMPINES WEST,1,2


In [82]:
df_merge.to_csv("mapequation_com.csv", index_label="ind")

In [15]:
data_dir = "../data"

In [18]:
day = "weekday" # "weekend"
fp = os.path.join(data_dir, "net_202001_{}.net".format(day))
dg1 = nx.read_pajek(fp)
dg1 = nx.DiGraph(dg1)
dg1.number_of_nodes(), dg1.number_of_edges()

(303, 30043)

In [19]:
day = "weekend"
fp2 = os.path.join(data_dir, "net_202001_{}.net".format(day))
dg2 = nx.read_pajek(fp2)
dg2 = nx.DiGraph(dg2)
dg2.number_of_nodes(), dg2.number_of_edges()

(303, 30043)

In [76]:
def conn_entropy(dg, df, col, direction="out", edge_weight="weight"):
    node_dic = {}
    for n, v in zip(df["node"].tolist(), df[col].tolist()):
        node_dic[n] = v
    entropy_dic = {}
    for n in dg.nodes():
        nb = None
        if direction=="out":
            nb = dg.out_edges(nbunch=n,data=True)
            nb = [ (v,d[edge_weight]) for u,v,d in nb ]
        else:
            nb = dg.in_edges(nbunch=n, data=True)
            nb = [ (u,d[edge_weight]) for u,v,d in nb ]
        if len(nb)<=1:
            entropy_dic[n] = -1
            continue
        tot_weight = sum([ w for c,w in nb ])
        partial = [ float(w)/tot_weight for c,w in nb ]
        cats = [ node_dic[c] for c,w in nb ]
        cat_set = list(set(cats))
        if len(cat_set)<=1:
            entropy_dic[n] = 0.
            continue
        proportions = { cat:0. for cat in cat_set }
        for cat,prop in zip(cats, partial):
            proportions[cat]+=prop
        ent = -sum([ p*np.log(p) for p in proportions.values() ])/np.log(len(cat_set))
        #print(ent)
        entropy_dic[n] = ent
        #break
    return entropy_dic

In [83]:
df_com = pd.read_csv("mapequation_com.csv", index_col=0)
df_com.head()

,node,level_1_weekday,level_1_weekend
ind,,,
0,TAMPINES EAST,1,2
1,BEDOK NORTH,1,2
2,TOA PAYOH CENTRAL,1,2
3,ALJUNIED,1,1
4,TAMPINES WEST,1,2


In [84]:
com_entropy_1_in = conn_entropy(dg1, df_com, "level_1_weekday", direction="in", edge_weight="weight")
com_entropy_1_out = conn_entropy(dg1, df_com, "level_1_weekday", direction="out", edge_weight="weight")
com_entropy_2_in = conn_entropy(dg2, df_com, "level_1_weekend", direction="in", edge_weight="weight")
com_entropy_2_out = conn_entropy(dg2, df_com, "level_1_weekend", direction="out", edge_weight="weight")
#check#["THE WHARVES"]

In [86]:
df_entropy = pd.DataFrame.from_dict({"weekday_com_ent_in":com_entropy_1_in, 
                                     "weekday_com_ent_out":com_entropy_1_out, 
                                     "weekend_com_ent_in":com_entropy_2_in, 
                                     "weekend_com_ent_out":com_entropy_2_out, 
                                     })
df_entropy.head()

,weekday_com_ent_in,weekday_com_ent_out,weekend_com_ent_in,weekend_com_ent_out
ADMIRALTY,0.364968,0.471079,0.390905,0.546073
AIRPORT ROAD,0.421583,0.269137,0.922025,0.964743
ALEXANDRA HILL,0.295339,0.320279,0.262960,0.320530
ALEXANDRA NORTH,0.260659,0.100250,0.215390,0.071056
ALJUNIED,0.401774,0.432890,0.293779,0.342999


In [87]:
com_res = pd.merge(df_com, df_entropy, left_on="node", right_index=True, suffixes=["_com", "entropy"])

In [88]:
com_res.head()

,node,level_1_weekday,level_1_weekend,weekday_com_ent_in,weekday_com_ent_out,weekend_com_ent_in,weekend_com_ent_out
ind,,,,,,,
0,TAMPINES EAST,1,2,0.353991,0.311314,0.294636,0.294559
1,BEDOK NORTH,1,2,0.229097,0.178567,0.346991,0.281375
2,TOA PAYOH CENTRAL,1,2,0.503925,0.463503,0.519752,0.497265
3,ALJUNIED,1,1,0.401774,0.432890,0.293779,0.342999
4,TAMPINES WEST,1,2,0.249632,0.302093,0.199284,0.215336


In [89]:
com_res.to_csv("mapequation_com_entropy.csv", index_label="ind")